In [55]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import lxml
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os

In [56]:
states = {
"25" : "ANDAMAN and NICOBAR","1 " : "ANDHRA PRADESH","22" : "ARUNACHAL PRADESH","2 " : "ASSAM",
"3 " : "BIHAR","26" : "CHANDIGARH","33" : "CHATTISGARH","30" : "DADAR and NAGAR HAVELI",
"31" : "DAMAN and DIU","27" : "DELHI","50" : "FOREIGN SCHOOLS","28" : "GOA",
"4 " : "GUJARAT","5 " : "HARYANA","6 " : "HIMACHAL PRADESH","7 " : "JAMMU and KASHMIR",
"34" : "JHARKHAND","8 " : "KARNATAKA","9 " : "KERALA","37" : "LADAKH",
"32" : "LAKSHADWEEP","10" : "MADHYA PRADESH","11" : "MAHARASHTRA","12" : "MANIPUR",
"13" : "MEGHALAYA","23" : "MIZORAM","14" : "NAGALAND","15" : "ODISHA",
"29" : "PUDUCHERRY","16" : "PUNJAB","17" : "RAJASTHAN","18" : "SIKKIM","19" : "TAMILNADU",
"36" : "TELANGANA","20" : "TRIPURA","21" : "UTTAR PRADESH","35" : "UTTARAKHAND","24" : "WEST BENGAL"}
chromedriver = r"C:\Users\jaske\Downloads\chromedriver_win32\chromedriver"

In [57]:
for k,v in states.items():
    url = "http://cbseaff.nic.in/cbse_aff/schdir_Report/userview.aspx"
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)

    radio = driver.find_element_by_xpath('//*[@id="optlist_2"]')
    radio.click()

    state = Select(driver.find_element_by_xpath('//*[@id="ddlitem"]'))
    state.select_by_value(k)

    #district = Select(driver.find_element_by_xpath('//*[@id="DropDownDistrict"]'))
    #district.select_by_value("30")

    search = driver.find_element_by_xpath('//*[@id="search"]')
    search.click()

    num = driver.find_element_by_xpath('//*[@id="lbltotal1"]').text
    
    try:
        os.remove(r"C:\Users\jaske\OneDrive\Desktop\data\data1.csv")
        os.remove(r"C:\Users\jaske\OneDrive\Desktop\data\data2.csv")
    except Exception as e:
        pass
    
    for i in range(1, int(num)//25 + 2):
        
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        
        content = soup.find("table", {"id":"T1"})
        main = content.find("tbody")
        tables = main.find_all("table")
        count = 0
        desktop = r"C:\Users\jaske\OneDrive\Desktop\data"
        path1 = desktop + "\data1.csv"
        path2 = desktop + "\data2.csv"
        for tab in tables[:][1:]:
            l = []
            count = count + 1
            tds = tab.find_all("td")
            for td in tds:
                l.append(td.text.replace("\t","").replace("\n","").replace(",",", "))
            if(count%3 == 1):
                pass
            else:
                if(count%3 == 0):
                    with open(path2, "a", newline = "") as fp:
                        wr = csv.writer(fp, dialect='excel')
                        wr.writerow(l)
                    fp.close()
                elif((count%2 == 0) or (count%2 == 1)):
                    with open(path1, "a", newline = "") as f:
                        wr = csv.writer(f, dialect='excel')
                        wr.writerow(l)
                    f.close()

        try:    
            nextbut = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,'//*[@id="Button1"]')))
            driver.execute_script("arguments[0].click();", nextbut)
        except Exception as e:
            pass
    driver.close()
    f.close()
    fp.close()

    df1 = pd.read_csv(path1, encoding='cp1252', header = None)
    df2 = pd.read_csv(path2, encoding='cp1252', header = None)
    df1.insert(0, 'Sr no.', range(0, len(df1)))
    df2.insert(0, 'Sr no.', range(0, len(df2)))
    df3 = pd.merge(df1,df2, on=['Sr no.'])
    df3.columns = ["Sr no.","Affiliation No.","Name","Head/Principal Name","Status of the School","Affiliated up to","Address","Phone No","Email"]
    df3[["Email","Website"]] = df3.Email.str.split("Website:",expand = True)
    for i in df3.columns[:-1]:
        try:
            df3[i] = df3[i].str.replace(i,"")
            df3[i] = df3[i].str.replace(":","")
        except Exception as e:
            pass
    df3.to_excel(desktop + "\\" + v + ".xlsx", index = None)